In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from tqdm import tqdm
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pickle
import tensorflow as tf
import math
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [10]:
""" One v.s. All binary classification """

' One v.s. All binary classification '

In [11]:
df = pd.read_csv("../Data_Scrape/Dataset/updated_dataset.csv")

In [12]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"


In [13]:
playlists_multi = [132, 123,19,46,60,68,79,125]  #132, 123,19,46,60,68,79,125
# playlists_multi = [i for i in range(1,169,1)]  #132, 123,19,46,60,68,79,125

dict_playlists_to_songs = {}
data = np.array(df)
Dataset = []
for j in playlists_multi:
    l0 = []
    for i in df.index:
        l = eval(df["Playlists"][i])
        if(j in l):
            l.remove(j)
            flag = 0
            for k in playlists_multi:
                    if(k!=j and k in l):
                        flag = 1
            if(flag!=1):
                l0.append(df["sp_song_id"][i])
                Dataset.append(data[i])
                
    dict_playlists_to_songs[j] = l0
#     print(len(l0))
print(len(dict_playlists_to_songs))
Dataset = np.array(Dataset)
print(Dataset.shape)
np.random.shuffle(Dataset)
print(Dataset[0])



8
(2547, 36)
['4FZoFK86X6Wd1csU9afTIA' 0.344 0.879 2 -6.546 1 0.0481 4.99e-05 0.875
 0.24100000000000002 0.371 118.814 214760 2009 37 0.0 0.04333582147955895
 0.0 0.0 0.0 0.0 0.2181074470281601 0.0 0.0 0.0 0.0 0.0
 0.01100169774144888 0.5925655364990234 0.0 0.0 0.0 0.06927943974733353
 0.0 0.05013835057616234 '[79]']


In [14]:
X = Dataset[:,1:-1]
Y = Dataset[:,-1]
# print(Y)
for i in range(len(Y)):
    Y[i] = eval(Y[i])
    for j in playlists_multi:
        if(j in Y[i]):
            Y[i] = j
            break
    
print(X.shape)
print(Y.shape)



(2547, 34)
(2547,)


In [ ]:
X = np.array(X)
y = np.array(Y)
y = Y.reshape(-1, 1)
standardScalar = StandardScaler()
X = standardScalar.fit_transform(X)
print(X.shape, y.shape)
y = np.squeeze(y)
# print(X)
# print(y)

In [ ]:
y = y.astype("str")
# print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne_em = TSNE(n_components=2, perplexity=30.0, n_iter=1000, verbose=1).fit_transform(X_train)
print(tsne_em)

In [ ]:
df_tsne= pd.DataFrame(columns = ["x", "y", "label"] )
df_tsne["x"] = tsne_em[:,0]
df_tsne["y"] = tsne_em[:,1]
df_tsne["label"] = y_train

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (20,20))
# plt.scatter(tsne_em[:,0], tsne_em[:,1],color = "green", hue = y_train);
sns.scatterplot(x = df_tsne["x"], y = df_tsne["y"], hue = df_tsne["label"],  palette = sns.color_palette("husl", len(set(y_train))))
plt.show()

In [ ]:

logistic = LogisticRegression(solver = "saga", multi_class = "multinomial")

logistic.fit(X_train, y_train)

test_score = logistic.score(X_test, y_test)

y_pred = logistic.predict(X_test)
pres_score = precision_score(y_test, y_pred, average = "weighted")

f1= f1_score(y_test, y_pred, average = "weighted")
recall_Score = recall_score(y_test, y_pred, average = "weighted")
print(test_score, pres_score, f1, recall_Score)

In [2]:
""" One vs One Binary classification"""

' One vs One Binary classification'

In [3]:
df = pd.read_csv("../Data_Scrape/Dataset/updated_dataset.csv")

In [4]:
df.head()

,sp_song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,lda_topic_11,lda_topic_12,lda_topic_13,lda_topic_14,lda_topic_15,lda_topic_16,lda_topic_17,lda_topic_18,lda_topic_19,Playlists
0,6edQfeOlqbGteYixpJl3Sm,0.857,0.602,10,-8.311,1,0.0290,0.489000,0.02440,0.3170,...,0.0,0.022993,0.000000,0.122078,0.0,0.000000,0.000000,0.081108,0.386042,[165]
1,5Oe7wHPL4hdEXeF4AOayCi,0.322,0.990,8,-1.785,1,0.1710,0.000044,0.41700,0.0366,...,0.0,0.000000,0.163142,0.022999,0.0,0.144614,0.187207,0.000000,0.000000,[79]
2,6i1uWZYWabNHq2wQnoca58,0.666,0.884,9,-5.243,0,0.0429,0.506000,0.00612,0.0408,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.582131,[15]
3,4BzBtS6PBreni5hNPo2hos,0.609,0.578,9,-7.081,1,0.0414,0.296000,0.00000,0.1500,...,0.0,0.000000,0.511716,0.084905,0.0,0.000000,0.000000,0.115004,0.108140,[168]
4,0GvhHQbWSnGltjl0je61dI,0.699,0.514,4,-12.610,1,0.0315,0.587000,0.02900,0.2200,...,0.0,0.000000,0.000000,0.401954,0.0,0.000000,0.000000,0.000000,0.000000,"[30, 133]"


In [16]:
playlists = []
result_history = {}

np.random.seed(123)

for playlist in tqdm(range(0,169,1), desc = "Progress : ", position = 0, leave = True):
    X = []
    y = []
    list_of_is = []
    for i in df.index:
        a = list(df.iloc[i])[1:-1]
        if playlist in eval(df['Playlists'][i]):
            X.append(a)
            y.append(1)
        else:
            _ = np.random.randint(0, 26)
            if _ == 0:
                X.append(a)
                y.append(0)
    
        
    
    X = np.array(X)
    y = np.array(y)
    


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify = y)
    
    standardScalar = StandardScaler()
    X_train = standardScalar.fit_transform(X_train)
    X_test = standardScalar.transform(X_test)
    
#     model = LogisticRegression()

    model = SVC(kernel = 'linear')
    
    param = {
#     'eta': 0.2, 
    'max_depth': 21,  
    'objective': 'multi:softprob',  
    'num_class': 2,
    'n_estimators': 500
#     'booster': 'dart',
#     'importance_type': "gain",
#     'gamma':1,
#     'subsample': 0.8
    } 

    xgb = XGBClassifier(**param)    
    
    model.fit(X_train, y_train)
    
#     print(len(X_train))
    
    y_pred = model.predict(X_test)
    pres_score = precision_score(y_test, y_pred, average = "macro")

    f1= f1_score(y_test, y_pred, average = "macro")
    recall_Score = recall_score(y_test, y_pred, average = "macro")
    
    result_history[playlist] = {"Precision Score " : pres_score, "Recall Score " : recall_Score, "F1 Score " : f1 }
    
#     print(pres_score, recall_Score, f1)
    
print(result_history)
   
    
        
        
        
    
    

Progress : 100%|█████████████████████████████████████████████████████████████████████| 169/169 [17:56<00:00,  6.37s/it]


{0: {'Precision Score ': 0.7782805429864253, 'Recall Score ': 0.7782805429864253, 'F1 Score ': 0.7782805429864253}, 1: {'Precision Score ': 0.8007246376811594, 'Recall Score ': 0.7850274725274725, 'F1 Score ': 0.7925260003525472}, 2: {'Precision Score ': 0.6242236024844721, 'Recall Score ': 0.6172161172161172, 'F1 Score ': 0.6205012247974374}, 3: {'Precision Score ': 0.7928571428571429, 'Recall Score ': 0.8327922077922078, 'F1 Score ': 0.8072562358276643}, 4: {'Precision Score ': 0.7301136363636364, 'Recall Score ': 0.7093023255813953, 'F1 Score ': 0.7187288708586883}, 5: {'Precision Score ': 0.7380392156862745, 'Recall Score ': 0.7955209347614411, 'F1 Score ': 0.761038961038961}, 6: {'Precision Score ': 0.8272727272727273, 'Recall Score ': 0.8009404388714734, 'F1 Score ': 0.8133333333333332}, 7: {'Precision Score ': 0.8175053153791637, 'Recall Score ': 0.9242424242424242, 'F1 Score ': 0.8588425085702762}, 8: {'Precision Score ': 0.7673913043478261, 'Recall Score ': 0.7277777777777779,

In [18]:
# f = open("binary_svc_linear_result", "wb")
# pickle.dump(result_history, f)
# f.close()

In [52]:
f = open("binary_svc_linear_result", "rb")
l = pickle.load(f)
f.close()

In [53]:
l = {k: v for k, v in sorted(l.items(), key=lambda item: item[1]["F1 Score "], reverse = True)}

In [54]:
# Top 20 playlists 
for key in list(l.keys())[:20]:
    print(key," = ",  l[key])

71  =  {'Precision Score ': 0.9615384615384616, 'Recall Score ': 0.9945652173913043, 'F1 Score ': 0.9772677595628416}
89  =  {'Precision Score ': 0.955977229601518, 'Recall Score ': 0.955977229601518, 'F1 Score ': 0.955977229601518}
137  =  {'Precision Score ': 0.9624482554701359, 'Recall Score ': 0.9443181818181818, 'F1 Score ': 0.953063885267275}
100  =  {'Precision Score ': 0.925, 'Recall Score ': 0.9836956521739131, 'F1 Score ': 0.9511721666417798}
99  =  {'Precision Score ': 0.929128277817151, 'Recall Score ': 0.929128277817151, 'F1 Score ': 0.929128277817151}
98  =  {'Precision Score ': 0.9108527131782946, 'Recall Score ': 0.9430512016718913, 'F1 Score ': 0.9261120884644383}
66  =  {'Precision Score ': 0.9211764705882353, 'Recall Score ': 0.9282296650717703, 'F1 Score ': 0.9245823860828308}
97  =  {'Precision Score ': 0.8999208860759493, 'Recall Score ': 0.9457671957671958, 'F1 Score ': 0.9208333333333333}
161  =  {'Precision Score ': 0.9332659251769464, 'Recall Score ': 0.908095

In [85]:
model = Sequential([
    Dense(128, activation = 'relu', input_shape = (34,)),
#     Dense(256, activation = 'relu'),
    Dense(169, activation = 'softmax')   
    
])
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [86]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 128)               4480      
_________________________________________________________________
dense_22 (Dense)             (None, 169)               21801     
Total params: 26,281
Trainable params: 26,281
Non-trainable params: 0
_________________________________________________________________


In [87]:
data = np.array(df)
X = data[:,1:-1]
Y = data[:,-1]
y = np.zeros((len(Y), 169))
for i,label in enumerate(Y):
    curr_label = eval(label)
    
    for playlist in curr_label:
        y[i,playlist] = 1
        break

In [88]:
print(X.shape)
print(y.shape)

(11159, 34)
(11159, 169)


In [89]:
X = np.asarray(X).astype('float32')

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [91]:
history = model.fit(X_train, y_train , validation_data = (X_test,y_test), batch_size = 256 , epochs = 100)

Epoch 1/100
35/35 [==============================] - ETA: 0s - loss: 65421.7109 - accuracy: 0.00 - ETA: 0s - loss: 45598.8867 - accuracy: 0.01 - 0s 6ms/step - loss: 40442.0117 - accuracy: 0.0115 - val_loss: 10069.4434 - val_accuracy: 0.0067
Epoch 2/100
35/35 [==============================] - ETA: 0s - loss: 10609.1152 - accuracy: 0.0000e+ - ETA: 0s - loss: 1582.8091 - accuracy: 0.0117     - 0s 3ms/step - loss: 1000.4699 - accuracy: 0.0115 - val_loss: 5.0873 - val_accuracy: 0.0121
Epoch 3/100
35/35 [==============================] - ETA: 0s - loss: 5.0827 - accuracy: 0.01 - ETA: 0s - loss: 5.0504 - accuracy: 0.01 - 0s 3ms/step - loss: 5.0395 - accuracy: 0.0117 - val_loss: 5.0004 - val_accuracy: 0.0112
Epoch 4/100
35/35 [==============================] - ETA: 0s - loss: 5.0023 - accuracy: 0.00 - ETA: 0s - loss: 4.9734 - accuracy: 0.01 - 0s 3ms/step - loss: 4.9635 - accuracy: 0.0130 - val_loss: 4.9324 - val_accuracy: 0.0121
Epoch 5/100
35/35 [==============================] - ETA: 0s - l

35/35 [==============================] - ETA: 0s - loss: 4.7334 - accuracy: 0.04 - ETA: 0s - loss: 4.6919 - accuracy: 0.05 - 0s 4ms/step - loss: 4.6927 - accuracy: 0.0535 - val_loss: 4.6928 - val_accuracy: 0.0587
Epoch 37/100
35/35 [==============================] - ETA: 0s - loss: 4.6472 - accuracy: 0.07 - ETA: 0s - loss: 4.6997 - accuracy: 0.05 - 0s 3ms/step - loss: 4.6925 - accuracy: 0.0535 - val_loss: 4.6928 - val_accuracy: 0.0587
Epoch 38/100
35/35 [==============================] - ETA: 0s - loss: 4.6918 - accuracy: 0.04 - ETA: 0s - loss: 4.6933 - accuracy: 0.05 - 0s 3ms/step - loss: 4.6926 - accuracy: 0.0535 - val_loss: 4.6928 - val_accuracy: 0.0587
Epoch 39/100
35/35 [==============================] - ETA: 0s - loss: 4.6906 - accuracy: 0.04 - ETA: 0s - loss: 4.6878 - accuracy: 0.05 - 0s 3ms/step - loss: 4.6925 - accuracy: 0.0535 - val_loss: 4.6926 - val_accuracy: 0.0587
Epoch 40/100
35/35 [==============================] - ETA: 0s - loss: 4.7065 - accuracy: 0.06 - ETA: 0s - los

Epoch 72/100
35/35 [==============================] - ETA: 0s - loss: 4.7343 - accuracy: 0.06 - ETA: 0s - loss: 4.7040 - accuracy: 0.05 - 0s 3ms/step - loss: 4.6919 - accuracy: 0.0535 - val_loss: 4.6925 - val_accuracy: 0.0587
Epoch 73/100
35/35 [==============================] - ETA: 0s - loss: 4.6622 - accuracy: 0.03 - ETA: 0s - loss: 4.6959 - accuracy: 0.04 - 0s 3ms/step - loss: 4.6917 - accuracy: 0.0535 - val_loss: 4.6922 - val_accuracy: 0.0587
Epoch 74/100
35/35 [==============================] - ETA: 0s - loss: 4.7511 - accuracy: 0.06 - ETA: 0s - loss: 4.6861 - accuracy: 0.05 - 0s 3ms/step - loss: 4.6919 - accuracy: 0.0535 - val_loss: 4.6928 - val_accuracy: 0.0587
Epoch 75/100
35/35 [==============================] - ETA: 0s - loss: 4.6141 - accuracy: 0.07 - ETA: 0s - loss: 4.6816 - accuracy: 0.05 - 0s 4ms/step - loss: 4.6918 - accuracy: 0.0535 - val_loss: 4.6929 - val_accuracy: 0.0587
Epoch 76/100
35/35 [==============================] - ETA: 0s - loss: 4.6006 - accuracy: 0.05 - 

In [70]:
y_pred = model.predict(X_test)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)